In [14]:
import os
import numpy as np
import pandas as pd
import tmunlp as nlp

import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras import optimizers
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import Dense, Embedding
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import concatenate, Bidirectional
from keras import initializers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM
from keras.activations import tanh, softmax
from keras.callbacks import ModelCheckpoint

In [11]:
def str_to_list(dataframe, string): 
    l = []
    n = []
    for i in dataframe[string]:
        if i != '[]':
            
            l.append([float(j) for j in str(i).split()])
        else:
            l.append(n)
    return l

# calculate length of the texts
def length(dataframe):
    len_num = []
    for i in dataframe['texts']:
        temp = i.replace(" ","")
        len_num.append(len(temp))
    
    dataframe['length'] = len_num
    return dataframe

from scipy import stats
import tensorflow as tf

def normalize(pred, truth):
    """ convert inputs to np.array and make sure
    inputs are normalized probability distributions
    """
    if len(pred) != len(truth):
        raise ValueError("pred and truth have different lengths")
    if len(pred) == 0 or len(truth) == 0:
        raise ValueError("pred or truth are empty")

    pred, truth = np.asarray(pred), np.asarray(truth)
    if not ((pred >= 0).all() and (truth >= 0).all()):
        raise ValueError("probability distribution should not be negative")
    pred, truth = pred / pred.sum(), truth / truth.sum()
    return pred, truth

def jensen_shannon_div(pred, truth, base=2):
    ''' JSD: Jensen-Shannon Divergence
    '''
    pred, truth = normalize(pred, truth)
    m = 1. / 2 * (pred + truth)
    return (stats.entropy(pred, m, base=base)
            + stats.entropy(truth, m, base=base)) / 2.

def root_normalized_squared_error(pred, truth):
    """ RNSS: Root Normalised Sum of Squares
    """

    def squared_error(pred, truth):
        return ((pred - truth) ** 2).sum()

    pred, truth = normalize(pred, truth)
    return np.sqrt(squared_error(pred, truth) / 2)

def jsd_custom_loss(y_true, y_pred):
            
    # calculate loss, using y_pred
    ''' JSD: Jensen-Shannon Divergence
    '''
#     y_pred, y_true = normalize(y_pred, y_true)
    m = 1. / 2 * (y_pred + y_true)
    # loss = (stats.entropy(y_pred, m, base=2) + stats.entropy(y_true, m, base=2)) / 2.
    # tf.keras.losses.KLD()
    loss = (tf.keras.losses.KLD(y_pred, m) + tf.keras.losses.KLD(y_true, m)) / 2.
    return loss
  

In [23]:
train = pd.read_csv('Dataset\\New_DialEval-1\\train_baidu_cn.csv', encoding='utf_8')
dev = pd.read_csv('Dataset\\New_DialEval-1\\dev_baidu_cn.csv', encoding='utf_8')

In [24]:
train['Baidu_str'] = train['Baidu_str'].fillna('[]')
dev['Baidu_str'] = dev['Baidu_str'].fillna('[]')
# test['W2V_str'] = test['W2V_str'].fillna('[]')

train["W2V"] = str_to_list(train, 'W2V_str')
dev["W2V"] = str_to_list(dev, 'W2V_str')
train = length(train)
dev = length(dev)
# test = length(test)


In [47]:
dev['round'].astype(str).unique()

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [25]:
def to_array(dataframe, length, string, dim=256):   
    train_vec = np.zeros(shape=(length,dim))
#     print(train_vec.shape)
    for i in range(len(dataframe[string])):
        if dataframe[string].iloc[i] != []:
#             print(type(dataframe[string].iloc[i]))
    #         print(i)
            train_vec[i] = dataframe[string].iloc[i]
        else:
            continue
    print(train_vec.shape)
    
    return train_vec

train_vec = to_array(train, len(train), 'W2V', dim=300)
dev_vec = to_array(dev, len(dev), 'W2V', dim=300)

(15400, 300)
(1755, 300)


In [26]:
train_vec = np.reshape(train_vec, (train_vec.shape[0],1,train_vec.shape[1]))
dev_vec = np.reshape(dev_vec, (dev_vec.shape[0],1,dev_vec.shape[1]))

In [39]:
X_train = train_vec
X_test = dev_vec
y_train = np_utils.to_categorical(train['round'].astype('str'))
y_test = np_utils.to_categorical(dev['round'].astype('str'))

In [43]:
# === customer
def lstm(X_train, X_test, y_train, y_test, loss='categorical_crossentropy'):
     
#     main_input = Input(shape=(350,), dtype='float64')
    main_input = Input(shape=(1,300))
    
#     embed = Embedding(output_dim=300,input_dim=20000,input_length=350)(main_input)
#     dropout_1 = Dropout(0.35)(embed)
    lst = Bidirectional(LSTM(units=256,return_sequences=True))(main_input)
    lst2 = Bidirectional(LSTM(units=128))(lst)
    dense_1 = Dense(units=64,activation='relu')(lst2)
    dropout_1 = Dropout(0.35)(dense_1)
#     dense_2 = Dense(units=32,activation='relu')(dropout_1)
    dense_3 = Dense(units=32,activation='relu')(dropout_1)
    output = Dense(units=8,activation='softmax')(dense_3)

    model = Model(inputs=main_input, outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    filepath="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights.best.hdf5"
    checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                                save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    train_history = model.fit(x=X_train, y=y_train, epochs=50, 
                              batch_size=128, callbacks=callbacks_list, verbose=1, validation_split=0.2)

    score = model.evaluate(x=X_test, y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=X_test)
    predicted = pre_probability.argmax(axis=-1)
    
    return model, train_history, pre_probability

In [44]:
lstm(X_train, X_test, y_train, y_test)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1, 300)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 1, 512)            1140736   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 256)               656384    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 264 

12320/12320 [==============================] - 1s 85us/step - loss: 0.7176 - accuracy: 0.7108 - val_loss: 0.9757 - val_accuracy: 0.6458

Epoch 00029: val_accuracy did not improve from 0.66818
Epoch 30/50
12320/12320 [==============================] - 1s 85us/step - loss: 0.7127 - accuracy: 0.7155 - val_loss: 0.9812 - val_accuracy: 0.6484

Epoch 00030: val_accuracy did not improve from 0.66818
Epoch 31/50
12320/12320 [==============================] - 1s 84us/step - loss: 0.6969 - accuracy: 0.7191 - val_loss: 1.0199 - val_accuracy: 0.6490

Epoch 00031: val_accuracy did not improve from 0.66818
Epoch 32/50
12320/12320 [==============================] - 1s 85us/step - loss: 0.6925 - accuracy: 0.7209 - val_loss: 1.0024 - val_accuracy: 0.6451

Epoch 00032: val_accuracy did not improve from 0.66818
Epoch 33/50
12320/12320 [==============================] - 1s 85us/step - loss: 0.6793 - accuracy: 0.7257 - val_loss: 1.0174 - val_accuracy: 0.6390

Epoch 00033: val_accuracy did not improve from 

(<keras.engine.training.Model at 0x1e645067860>,
 array([[5.80470776e-11, 9.89836514e-01, 5.90050968e-06, ...,
         1.56676420e-03, 1.24058595e-08, 1.99300965e-04],
        [2.01853003e-23, 2.53057562e-17, 1.18084520e-09, ...,
         8.61429393e-01, 1.79465201e-06, 1.31463923e-06],
        [2.27640612e-05, 1.33564323e-03, 1.76081732e-02, ...,
         1.28952667e-01, 5.23073366e-03, 9.26725566e-02],
        ...,
        [0.00000000e+00, 1.81692246e-29, 4.80833272e-20, ...,
         9.99903083e-01, 5.26479108e-19, 2.80414326e-07],
        [4.74133265e-13, 2.45832417e-11, 2.23361119e-03, ...,
         5.78574145e-06, 2.14238018e-01, 1.22401627e-08],
        [1.05023171e-26, 1.65784877e-12, 1.23444083e-12, ...,
         9.52615023e-01, 7.43816494e-14, 3.67099093e-03]], dtype=float32))

In [55]:
labels = {} # label : freq
label_wordlist = {}
dataset = [] # label : text
label_list=['1', '2', '3', '4', '5', '6', '7']    

data = train.filter(['round', 'texts'])

temp = []
for i in data['texts']:
    temp.append(i.split())
data['texts'] = temp
data['round'] = data['round'].astype(str)
data = data.values.tolist()

for l in label_list:
    labels[l] = 0
    label_wordlist[l] = dict()


for d in data:
    items = d

    if len(items) == 2:
        label = items[0]
        distinct_text = items[1]
#         seg_content = list(jieba.cut(text, cut_all = False))

#         distinct_text = list(set(seg_content)) #distinct the list, but revert the order
        dataset.append([label, distinct_text]) 

        labels[label] += 1

        for word in distinct_text:
            for l in label_list:
                if word in label_wordlist[l]:
                    label_wordlist[l][word].count_freq()
                else:
                    label_wordlist[l][word] = nlp.Feature(word)

In [60]:
type(label_wordlist)

dict

In [58]:
label_term_weighting = {}
for label, num_data_label in labels.items():
    for word in label_wordlist[label]:
        label_wordlist[label][word].reset_info()
    
    label_term_weighting[label] = nlp.get_term_weighting(label, num_data_label, label_wordlist[label], dataset)  

ValueError: math domain error

In [ ]:
def get_keyword(label, label_term_weighting, topN=10):
    keyword_list = {}
    for i in range(len(label_term_weighting[label])):
        if label_term_weighting[label][i][1]._present_on_topic > label_term_weighting[label][i][1]._present_off_topic:
            keyword_list[label_term_weighting[label][i][0]] = label_term_weighting[label][i][1]._llr
        if len(keyword_list) >= topN:
            return keyword_list
        
for key, value in get_keyword("sports", label_term_weighting, 5).items():
    print(key, value)